In [8]:
from collections import Counter
import numpy as np

In [2]:
def decisionz(values):
    
    # DECISIONS CONTAINER & THRESHOLD FOR DECISION
    decisions = []
    threshold = 0.02
    
    # LOOP THROUGH LOG VALUES
    for value in values:
        
        # BUY 
        if value > threshold:
            decisions.append('buy')
            
        # SELL
        elif value < -threshold:
            decisions.append('sell')
            
        # HOLD
        else:
            decisions.append('hold')
    
    return decisions

In [3]:
def morph(dataframe):
    for index, column in enumerate(dataframe.columns):
    
        # NEW COLUMN NAME
        name = column + '_log'
        
        # CALCULATE LOG VALUE BY CURRENT & NEXT
        dataframe[name] = decisionz(np.log(dataframe[column] / dataframe[column].shift(1)))
        
    # DROP NAN & SORT ALPHABETICALLY, THEN RETURN
    return dataframe.dropna()

In [30]:
def decide(columns, dataframe):
    
    # PLACEHOLDER
    decisions = []
    
    # FIND THE POPULAR VOTE FOR EACH ROW
    for index, row in dataframe[columns].iterrows():
        gather = Counter(list(row))
        vote = gather.most_common()[0][0]

        # APPEND POPULAR VOTE
        decisions.append(vote)
        
    # DROP DECISION COLS
    dataframe.drop([*columns, 'label'], axis=1, inplace=True)
    
    # RENAME THE LABELS COLUMN
    dataframe = dataframe.rename(columns={
        'label_log': 'label'
    })
    
    # ADD DECISIONS
    dataframe['united_decision'] = decisions
    
    # ADD BUY, SELL & HOLD COLUMNS
    dataframe['buy'] = 0
    dataframe['sell'] = 0
    dataframe['hold'] = 0
    
    # LOOP THROUGH DATAFRAME ROWS
    for index, row in dataframe.iterrows():

        #foob = row['decision'] 
        #dataframe.set_value(index, row['decision'], 1)
        choice = row['united_decision']
        dataframe.at[index, choice] = 1
    
    return dataframe

In [15]:
def classify(dataframe):
    
    # CLONE THE GIVEN DATAFRAME
    clone = dataframe.copy()
    
    # MORPH IN MODEL DEICISONS
    foo = morph(clone)
    
    # FOO
    bar = decide(['ensemble_log', 'linreg_0_log', 'lstm_1_log', 'lstm_2_log'], foo)
    
    return bar